# Plot all results from statistical option analysis

Make sure to create all relevant csv files in statAnalysis notebooks first

In [1]:
import os
# set working directory to MarketGenerators folder
# if you are working on LRZ servers, create the folder "MarketGenerators" and then specify something like
path = "/dss/dsshome1/02/YOUR_LRZ_USER_NAME/MarketGenerators"
os.chdir(path)

In [2]:
!pip install matplotlib

In [3]:
import pandas as pd
from src.visualization.plot_option_results import option_csv_plotting

In [ ]:
model_folders = [
    "GBM", 
    "Kou_Jump_Diffusion",
    "YFinance"
]
base_dir = "numerical_results"
option_types = ["call", "put"]
n_days_list = [5, 10, 21, 252]
bar_width = 0.26
# Strike price to evaluate
strike = "1.00"
relevant_year_folders= [
    "n-in=0Y", # YFinance
    "n-in=2Y", "n-in=9Y", "n-in=20Y", "n-in=50Y", "n-in=99Y", "n-in=999Y", # Retrained
    "n-in=3Y", "n-in=10Y", "n-in=100Y", "n-in=1000Y", "n-in=10000Y" # One-time trained
]
# Set ylim for each n_days (in %)
ylim_map = {
    5: (0, 52),
    10: (0, 52),
    21: (0, 62),
    252: (0, 52)
}
# Label for y-axis
price_label_map = {
    "GBM": "Exact",
    "Kou_Jump_Diffusion": "Approx.",
    "YFinance": "Input"
}
target_folder = "statistical_analysis_plots"

In [ ]:
for model_folder in model_folders:
    model_path = os.path.join(base_dir, model_folder)
    print(f"Start evaluation of {model_folder} based generators...")
    name_additions =  ["retrained_model_","same_model_"] if model_folder=="YFinance" else [""]
    price_label = price_label_map.get(model_folder, "?")

    for root, dirs, files in os.walk(model_path):
        years_folder_name = os.path.basename(root)
        if years_folder_name not in relevant_year_folders:
            continue
        print(f"   ...{root}")
        for option_type in option_types:
            for n_days in n_days_list:
                ylim = ylim_map.get(n_days, None)
                for name_addition in name_additions:
                    csv_file = os.path.join(root, f"{name_addition}summary_{option_type}_NDays={n_days}.csv")
                    df = pd.read_csv(csv_file)
                    file_name=f"{target_folder}/{model_folder}_{root.split('/')[2]}_{years_folder_name}_{n_days}_days_{option_type}{name_addition}"
                    option_csv_plotting(
                        df, strike=strike, option_type=option_type, price_label=price_label,
                        file_name=file_name, bar_width=bar_width, ylim=ylim
                    )
print("Done.")